In [31]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import time
import load_cifar
from sklearn.metrics import roc_auc_score, accuracy_score
from scipy.sparse import csr_matrix

In [32]:
tf.reset_default_graph()

In [33]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    data_shuffle = np.asarray(data_shuffle)
    labels_shuffle = np.asarray(labels_shuffle)
    labels_shuffle = np.squeeze(labels_shuffle)
    return data_shuffle, labels_shuffle

<h1>Define Hyper-perparmeter</h1>

In [34]:
num_classes = 10
num_features = 3072
num_output = 10
num_layers_0 = 500
num_layers_1 = 40
starter_learning_rate = 0.1
regularizer_rate = 0
batch_size = 2056
epochs = 100
dropout_prob = 0.8
training_accuracy = []
validation_accuracy = []
training_loss = []
testing_accuracy = []

In [35]:
#### EXAMPLE, FEEL FREE TO CHANGE #####
valid_features, valid_labels = pickle.load(open('vali_data.pickle','rb'))
training_features,training_labels = pickle.load(open('train_data.pickle','rb'))
test_features, test_labels = pickle.load(open('test_data.pickle','rb'))

In [36]:
valid_labels = np.squeeze(valid_labels.todense())
training_labels = np.squeeze(training_labels.todense())
test_labels = np.squeeze(test_labels.todense())

In [37]:
print(training_labels.shape)
print(training_labels[0])
print(training_features.shape)

(40000, 10)
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
(40000, 3072)


In [38]:
print(training_features[0,0:100])

[0.1372549  0.10588235 0.09803922 0.10196078 0.09411765 0.08627451
 0.09019608 0.09803922 0.09803922 0.09411765 0.10196078 0.09803922
 0.30980392 0.45098039 0.44313725 0.45490196 0.45882353 0.4627451
 0.49019608 0.50980392 0.44705882 0.42745098 0.4745098  0.47843137
 0.54117647 0.49411765 0.54117647 0.55686275 0.50588235 0.51764706
 0.52156863 0.49411765 0.24705882 0.1254902  0.06666667 0.08235294
 0.09411765 0.09019608 0.09019608 0.09019608 0.09019608 0.09019608
 0.09019608 0.08627451 0.36078431 0.50980392 0.48627451 0.49803922
 0.54117647 0.54117647 0.42352941 0.4627451  0.5372549  0.5254902
 0.52941176 0.54117647 0.5372549  0.51372549 0.52941176 0.4745098
 0.43137255 0.4        0.41176471 0.39215686 0.38823529 0.19215686
 0.05882353 0.07843137 0.09411765 0.09411765 0.09019608 0.09803922
 0.10980392 0.10980392 0.09803922 0.10196078 0.39215686 0.5254902
 0.46666667 0.42745098 0.53333333 0.54117647 0.39215686 0.4627451
 0.56078431 0.54901961 0.57254902 0.60392157 0.5254902  0.55294118


<h1>Define Placeholder</h1>

In [39]:
# Placeholders for the input data
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_y = tf.placeholder('float32',shape = (None,num_classes),name="input_Y")
## for dropout layer
keep_prob = tf.placeholder(tf.float32)


In [40]:
## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_classes], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_classes]))

<h1>Define Neural Network Architecture</h1>

In [41]:
## Initializing weigths and biases
hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0) + bias_0)
hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0_0,weights_1) + bias_1)
hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
predicted_y = tf.matmul(hidden_output_1_1,weights_2) + bias_2

<h1>Define cost and optimization</h1>

In [42]:
## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_y,labels=input_y)) \
        + regularizer_rate*(tf.reduce_sum(tf.square(bias_0)) + tf.reduce_sum(tf.square(bias_1)) \
                           + tf.reduce_sum(tf.square(bias_2)))
## Variable learning rate
#learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
learning_rate = starter_learning_rate
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct_prediction = tf.equal(tf.argmax(input_y,1), tf.argmax(predicted_y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [43]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for epoch in range(epochs): 
        start_time = time.time()
        for index in range(0,60000,batch_size):
            batch_x, batch_y = next_batch(batch_size, training_features, training_labels)
            s.run(optimizer, feed_dict={input_X:batch_x, input_y:batch_y, keep_prob:dropout_prob})
        use_time = time.time() - start_time
        training_accuracy.append(s.run(accuracy, feed_dict= {input_X:batch_x, 
                                                             input_y:batch_y,keep_prob:1}))
        training_loss.append(s.run(loss, {input_X: batch_x, 
                                          input_y: batch_y,keep_prob:1}))

        # Evaluation of model
        validation_accuracy.append(s.run(accuracy, feed_dict= {input_X:valid_features, \
                                                               input_y:valid_labels,keep_prob:1}))
        #print(s.run(accuracy, feed_dict= {input_X:valid_features, input_y:valid_labels,keep_prob:1}))
        print("Epoch:{0}, Train time: {1:.3f} Train loss: {2:.2f} Train acc: {3:.3f}, \
        Validation acc:{4:.3f}".format(epoch, use_time, training_loss[epoch], training_accuracy[epoch], \
                              validation_accuracy[epoch]))
    test_accuracy = accuracy_score(test_labels.argmax(1), s.run(predicted_y, {input_X: test_features, keep_prob:1}).argmax(1))
    print('testing accuracy is: ', test_accuracy)

Epoch:0, Train time: 2.503 Train loss: 2.50 Train acc: 0.104,         Validation acc:0.098
Epoch:1, Train time: 2.420 Train loss: 2.31 Train acc: 0.089,         Validation acc:0.103
Epoch:2, Train time: 2.462 Train loss: 2.30 Train acc: 0.104,         Validation acc:0.100
Epoch:3, Train time: 2.094 Train loss: 2.30 Train acc: 0.102,         Validation acc:0.102
Epoch:4, Train time: 2.430 Train loss: 2.30 Train acc: 0.092,         Validation acc:0.101
Epoch:5, Train time: 2.415 Train loss: 2.30 Train acc: 0.105,         Validation acc:0.097
Epoch:6, Train time: 2.407 Train loss: 2.30 Train acc: 0.104,         Validation acc:0.100
Epoch:7, Train time: 2.586 Train loss: 2.30 Train acc: 0.110,         Validation acc:0.103
Epoch:8, Train time: 2.422 Train loss: 2.30 Train acc: 0.098,         Validation acc:0.102
Epoch:9, Train time: 2.006 Train loss: 2.30 Train acc: 0.096,         Validation acc:0.102
Epoch:10, Train time: 2.382 Train loss: 2.30 Train acc: 0.097,         Validation acc:0.10

Epoch:90, Train time: 2.355 Train loss: 2.30 Train acc: 0.110,         Validation acc:0.101
Epoch:91, Train time: 2.313 Train loss: 2.30 Train acc: 0.102,         Validation acc:0.102
Epoch:92, Train time: 2.131 Train loss: 2.30 Train acc: 0.105,         Validation acc:0.103
Epoch:93, Train time: 2.101 Train loss: 2.30 Train acc: 0.095,         Validation acc:0.094
Epoch:94, Train time: 2.455 Train loss: 2.30 Train acc: 0.102,         Validation acc:0.100
Epoch:95, Train time: 2.498 Train loss: 2.31 Train acc: 0.104,         Validation acc:0.103
Epoch:96, Train time: 2.384 Train loss: 2.30 Train acc: 0.105,         Validation acc:0.103
Epoch:97, Train time: 2.495 Train loss: 2.30 Train acc: 0.096,         Validation acc:0.101
Epoch:98, Train time: 2.246 Train loss: 2.30 Train acc: 0.099,         Validation acc:0.102
Epoch:99, Train time: 2.100 Train loss: 2.30 Train acc: 0.105,         Validation acc:0.103
testing accuracy is:  0.1
